In [3]:
# Start Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("News Sentiment Analysis").getOrCreate()


In [4]:

# Fetch News from NewsAPI
import requests
api_key = "fb9d74dc3ef14050b8a6be2148b10f2b"  # Replace with your actual key
query = "Artificial Intelligence"
url = f"https://newsapi.org/v2/everything?q={query}&language=en&pageSize=100&apiKey={api_key}"

response = requests.get(url)
articles = response.json().get("articles", [])



In [5]:
# Prepare Data
from pyspark.sql import Row
rows = [Row(title=art['title'], description=art['description'] or "") for art in articles]
news_df = spark.createDataFrame(rows)


In [6]:
# Text Preprocessing & Sentiment Analysis
from textblob import TextBlob
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def get_sentiment(text):
    if not text: return "neutral"
    score = TextBlob(text).sentiment.polarity
    return "positive" if score > 0 else "negative" if score < 0 else "neutral"

sentiment_udf = udf(get_sentiment, StringType())
news_df = news_df.withColumn("sentiment", sentiment_udf(news_df.description))


In [7]:
# Show Sentiment Summary
news_df.groupBy("sentiment").count().show()

# Show Top Positive/Negative Headlines
print("🔹 Positive News:")
news_df.filter("sentiment = 'positive'").select("title").show(5, truncate=False)

print("🔹 Negative News:")
news_df.filter("sentiment = 'negative'").select("title").show(5, truncate=False)

# Stop Spark (for local runs)
# spark.stop()

+---------+-----+
|sentiment|count|
+---------+-----+
| positive|   40|
|  neutral|   18|
| negative|   42|
+---------+-----+

🔹 Positive News:
+----------------------------------------------------------------------------------------------------------+
|title                                                                                                     |
+----------------------------------------------------------------------------------------------------------+
|Delta Air Lines is using AI to set the maximum price you’re willing to pay                                |
|Andy Cohen on How He Uses AI: ‘It’s an Incredible Tool. But It’s Going to Make Idiots’                    |
|Microsoft transfers a top cybersecurity executive out of the company's security group, internal memo shows|
|Artificial intelligence fuels Big Tech partnerships with nuclear energy producers                         |
|The Latest ‘Alien: Earth’ Trailer Adds Even More Alien Monsters                             

In [8]:
news_df.toPandas().to_csv("news_sentiment.csv", index=False)
